# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 10 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 6 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - resources/lab2/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [6]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (10 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [7]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    texts = []
    for i in range(len(train_text)):
        texts.append(train_text[i].split())
    print(texts[0])

    for i in range(len(texts)):
        if bos == True:
            texts[i].insert(0, '<s>')
        if eos == True:
            texts[i].append('</s>')
    o = order
    '''
    for m in range(len(texts)):
        for word in texts[m]:
            ngrams[(word,)] += 1
        if order > 1:
            for i in range(len(texts[m])):
                try:
                    ngrams[(texts[m][i], texts[m][i:i+order])] += 1
                except:
                    break
    '''
    for m in range(len(texts)):
        for n in range(1, order + 1):
            for i in range(len(texts[m]) - n + 1):
                ngram = tuple(texts[m][i:i+n])
                ngrams[ngram] += 1


    print(dict(ngrams))
    return dict(ngrams)

In [8]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

['привет', 'привет', 'как', 'дела']
{('<s>',): 1, ('привет',): 2, ('как',): 1, ('дела',): 1, ('</s>',): 1}
['привет', 'привет', 'как', 'дела']
{('привет',): 2, ('как',): 1, ('дела',): 1, ('</s>',): 1}
['привет', 'привет', 'как', 'дела']
{('привет',): 2, ('как',): 1, ('дела',): 1}
['привет', 'привет', 'как', 'дела']
{('привет',): 2, ('как',): 1, ('дела',): 1, ('привет', 'привет'): 1, ('привет', 'как'): 1, ('как', 'дела'): 1}
['привет', 'привет', 'привет', 'привет', 'привет', 'привет']
{('привет',): 6, ('привет', 'привет'): 5}
['практическое', 'сентября']
{('<s>',): 3, ('практическое',): 2, ('сентября',): 2, ('</s>',): 3, ('<s>', 'практическое'): 1, ('практическое', 'сентября'): 1, ('сентября', '</s>'): 1, ('<s>', 'практическое', 'сентября'): 1, ('практическое', 'сентября', '</s>'): 1, ('<s>', 'практическое', 'сентября', '</s>'): 1, ('второе',): 1, ('занятие',): 1, ('пройдет',): 1, ('в',): 4, ('офлайне',): 2, ('32',): 3, ('12',): 2, ('часов',): 1, ('минуты',): 1, ('<s>', 'второе'): 1, ('


Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - add-k сглаживание ([Additive, Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [9]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    # TODO
    if V is None:
        # Вычисляем размер словаря как количество уникальных слов
        V = len(set(gram[0] for gram in counts.keys() if len(gram) == 1))
    
    if len(ngram) == 1:
        # Для униграмм: P(w) = count(w) / total_words
        total_words = sum(count for gram, count in counts.items() if len(gram) == 1)
        count_ngram = counts.get(ngram, 0)
        return (count_ngram + k) / (total_words + k * V)
    else:
        # Для n-грамм: P(w_i|context) = count(ngram) / count(context)
        context = ngram[:-1]
        count_ngram = counts.get(ngram, 0)
        count_context = counts.get(context, 0)
        prob =  (count_ngram + k) / (count_context + k * V)
    return prob

In [10]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

['практическое', 'сентября']
{('<s>',): 3, ('практическое',): 2, ('сентября',): 2, ('</s>',): 3, ('<s>', 'практическое'): 1, ('практическое', 'сентября'): 1, ('сентября', '</s>'): 1, ('<s>', 'практическое', 'сентября'): 1, ('практическое', 'сентября', '</s>'): 1, ('<s>', 'практическое', 'сентября', '</s>'): 1, ('второе',): 1, ('занятие',): 1, ('в',): 4, ('офлайне',): 2, ('32',): 3, ('12',): 2, ('часов',): 1, ('минуты',): 1, ('<s>', 'второе'): 1, ('второе', 'практическое'): 1, ('практическое', 'занятие'): 1, ('занятие', 'в'): 1, ('в', 'офлайне'): 2, ('офлайне', '32'): 1, ('32', 'сентября'): 1, ('сентября', 'в'): 1, ('в', '12'): 1, ('12', 'часов'): 1, ('часов', '32'): 1, ('32', 'минуты'): 1, ('минуты', '</s>'): 1, ('<s>', 'второе', 'практическое'): 1, ('второе', 'практическое', 'занятие'): 1, ('практическое', 'занятие', 'в'): 1, ('занятие', 'в', 'офлайне'): 1, ('в', 'офлайне', '32'): 1, ('офлайне', '32', 'сентября'): 1, ('32', 'сентября', 'в'): 1, ('сентября', 'в', '12'): 1, ('в', '12', 

Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [16]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        self.ngrams_count = count_ngrams(train_text, order=self.order, bos=self.bos, eos=self.eos)
        
        # Построение словаря
        if self.vocab is None:
            self.vocab = set()
            for text in train_text:
                words = text.split()
                self.vocab.update(words)
            if self.bos:
                self.vocab.add('<s>')
            if self.eos:
                self.vocab.add('</s>')
        
        # Подсчет общего количества униграмм
        self._total_unigrams = sum(count for gram, count in self.ngrams_count.items() if len(gram) == 1)
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        # считаем логарифм вероятности конкретной нграмы
        if len(ngram) == 1:
            # Unigram case
            count_ngram = self.ngrams_count.get(ngram, 0)
            prob = (count_ngram + self.k) / (self._total_unigrams + self.k * self.V)
        else:
            # Higher order n-gram
            context = ngram[:-1]
            count_ngram = self.ngrams_count.get(ngram, 0)
            count_context = self.ngrams_count.get(context, 0)
            prob = (count_ngram + self.k) / (count_context + self.k * self.V)
        
        return np.log(prob) if prob > 0 else -np.inf
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO
        for i in range(len(words)):
            # Определяем контекст
            context_start = max(0, i - self.order + 1)
            context = words[context_start:i]
            ngram = tuple(context + [words[i]])
            
            logprob += self.predict_ngram_log_proba(ngram)
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        return logprob
        
    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        #
        if not text or text == ['']:
            words = []
        else:
            words = text[0].split() if isinstance(text[0], str) else text
            
        logprob = self.predict_log_proba(words)
        
        # Calculate number of words (for normalization)
        N = len(words)
        if self.bos:
            N += 1
        if self.eos:
            N += 1
            
        if N == 0:
            return float('inf')
            
        perplexity = np.exp(-logprob / N)
        return perplexity

In [17]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

['по-моему', 'мы', 'сэкономим', 'уйму', 'времени', 'если', 'я', 'сойду', 'с', 'ума', 'прямо', 'сейчас']
{('<s>',): 3, ('по-моему',): 3, ('мы',): 3, ('сэкономим',): 3, ('уйму',): 3, ('времени',): 3, ('если',): 3, ('я',): 3, ('сойду',): 3, ('с',): 3, ('ума',): 3, ('прямо',): 2, ('сейчас',): 3, ('</s>',): 3, ('<s>', 'по-моему'): 1, ('по-моему', 'мы'): 2, ('мы', 'сэкономим'): 3, ('сэкономим', 'уйму'): 3, ('уйму', 'времени'): 3, ('времени', 'если'): 2, ('если', 'я'): 3, ('я', 'сойду'): 2, ('сойду', 'с'): 3, ('с', 'ума'): 3, ('ума', 'прямо'): 2, ('прямо', 'сейчас'): 2, ('сейчас', '</s>'): 1, ('<s>', 'если'): 1, ('сейчас', 'по-моему'): 1, ('времени', '</s>'): 1, ('<s>', 'мы'): 1, ('я', 'сейчас'): 1, ('сейчас', 'сойду'): 1, ('ума', 'по-моему'): 1, ('по-моему', '</s>'): 1}


# 2. Предсказания с помощью языковой модели (6 балла)

In [18]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix
    context = prefix[-(lm.order-1):] if len(prefix) >= lm.order-1 else prefix
    
    candidates = []
    for word in lm.vocab:
        if word in ['<s>', '</s>']:
            continue
            
        ngram = tuple(context + [word])
        log_prob = lm.predict_ngram_log_proba(ngram)
        candidates.append((word, log_prob))
    
    # Сортируем по убыванию вероятности
    candidates.sort(key=lambda x: x[1], reverse=True)
    word = candidates[:topk]
    return word
    

Попробуйте обучить ngram языковую модель на нескольких стихотворениях. Не забудьте трансформировать стихотворение в удобный для ngram модели формат (как сделать так, чтобы модель моделировала рифму?). 
Попробуйте сгенерировать продолжение для стихотворения с помощью такой языковой модели. 

In [19]:
#Your code here
# Пример с стихотворениями
poems = [
    "я помню чудное мгновенье передо мной явилась ты",
    "мороз и солнце день чудесный еще ты дремлешь друг прелестный",
    "люблю грозу в начале мая когда весенний первый гром",
    "зима недаром злится прошла ее пора весна в окно стучится"
]

# Обучаем модель
lm = NgramLM(order=2, k=0.5)
lm.fit(poems)

# Предсказываем продолжение
prefix = ["люблю", "грозу"]
predictions = predict_next_word(lm, prefix, topk=3)
print("Продолжение для 'люблю грозу':")
for word, log_prob in predictions:
    print(f"  {word}: {np.exp(log_prob):.4f}")

# Вычисляем перплексию
test_poem = ["зима", "недаром", "злится"]
ppl = lm.ppl(test_poem)
print(f"Перплексия для тестового текста: {ppl:.2f}")

['я', 'помню', 'чудное', 'мгновенье', 'передо', 'мной', 'явилась', 'ты']
{('<s>',): 4, ('я',): 1, ('помню',): 1, ('чудное',): 1, ('мгновенье',): 1, ('передо',): 1, ('мной',): 1, ('явилась',): 1, ('ты',): 2, ('</s>',): 4, ('<s>', 'я'): 1, ('я', 'помню'): 1, ('помню', 'чудное'): 1, ('чудное', 'мгновенье'): 1, ('мгновенье', 'передо'): 1, ('передо', 'мной'): 1, ('мной', 'явилась'): 1, ('явилась', 'ты'): 1, ('ты', '</s>'): 1, ('мороз',): 1, ('и',): 1, ('солнце',): 1, ('день',): 1, ('чудесный',): 1, ('еще',): 1, ('дремлешь',): 1, ('друг',): 1, ('прелестный',): 1, ('<s>', 'мороз'): 1, ('мороз', 'и'): 1, ('и', 'солнце'): 1, ('солнце', 'день'): 1, ('день', 'чудесный'): 1, ('чудесный', 'еще'): 1, ('еще', 'ты'): 1, ('ты', 'дремлешь'): 1, ('дремлешь', 'друг'): 1, ('друг', 'прелестный'): 1, ('прелестный', '</s>'): 1, ('люблю',): 1, ('грозу',): 1, ('в',): 2, ('начале',): 1, ('мая',): 1, ('когда',): 1, ('весенний',): 1, ('первый',): 1, ('гром',): 1, ('<s>', 'люблю'): 1, ('люблю', 'грозу'): 1, ('грозу

In [20]:
def generate_text(lm: NgramLM, start_words: List[str], length=10):
    generated = start_words.copy()
    
    for _ in range(length):
        next_word_candidates = predict_next_word(lm, generated, topk=1)
        if not next_word_candidates:
            break
        next_word = next_word_candidates[0][0]
        generated.append(next_word)
        
    return ' '.join(generated)

# Генерация текста
generated = generate_text(lm, ["люблю"], length=8)
print(f"Сгенерированный текст: {generated}")

Сгенерированный текст: люблю грозу в начале мая когда весенний первый гром
